In [1]:
#MODULES
import pandas as pd  #we need pandas for dataframes,  pip install pandas
import numpy as np

import seaborn as sns
import plotly.express as px  #we need ploty for graphs, pip install plotly-express

import sqlite3  #Not in use

## <span style="color:darkviolet">**Dataframes**

In [2]:
csv_activity = pd.read_csv('CSV/activity.csv')
csv_address = pd.read_csv('CSV/address.csv',low_memory=False)
csv_branch = pd.read_csv('CSV/branch.csv')
csv_code = pd.read_csv('CSV/code.csv')
csv_contact = pd.read_csv('CSV/contact.csv')
csv_denomination = pd.read_csv('CSV/denomination.csv')
csv_enterprise = pd.read_csv('CSV/enterprise.csv')
csv_establishment = pd.read_csv('CSV/establishment.csv')
csv_meta = pd.read_csv('CSV/meta.csv')

In [3]:
df_activity = csv_activity
df_address = csv_address
df_branch = csv_branch
df_code = csv_code
df_contact = csv_contact
df_denomination = csv_denomination
df_enterprise = csv_enterprise
df_establishment = csv_establishment
df_meta = csv_meta

## <span style="color:darkviolet">**Questions**

In [4]:
df_enterprise['PercentageQ1'] = 1 / df_enterprise.shape[0]

#### <span style="color:darkviolet">**Q1 Which percentage of the companies are under which juridical form?**

In [5]:
df_q1 =  df_enterprise.groupby('JuridicalForm').sum().sort_values(by='PercentageQ1',ascending=False)

/tmp/ipykernel_10762/409500052.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_q1 =  df_enterprise.groupby('JuridicalForm').sum().sort_values(by='PercentageQ1',ascending=False)


In [6]:
df_JuridicalForm = df_code.loc[(df_code['Category'] == 'JuridicalForm') & (df_code['Language'] == 'FR')]
df_JuridicalForm = df_JuridicalForm.rename(columns ={'Category':'Category','Code':'JuridicalForm','Language':'Language','Description':'Description'})
df_JuridicalForm = df_JuridicalForm.drop(columns=['Category','Language'],axis=1)
df_JuridicalForm['JuridicalForm'] = df_JuridicalForm['JuridicalForm'].astype(int)

In [7]:
df_q1 = pd.merge(df_q1, df_JuridicalForm,
    how='left', on='JuridicalForm')

In [8]:
df_q1 = df_q1[['JuridicalForm','Description','PercentageQ1']]
df_q1

,JuridicalForm,Description,PercentageQ1
0,15.0,Société privée à responsabilité limitée,1.913329e-01
1,610.0,Société à responsabilité limitée,9.156214e-02
2,17.0,Association sans but lucratif,7.533709e-02
3,14.0,Société anonyme,5.315055e-02
4,70.0,Association des copropriétaires,5.045640e-02
...,...,...,...
102,127.0,Monts-de-Piété,5.366845e-07
103,373.0,Ministère des Classes moyennes,5.366845e-07
104,387.0,Ministère de la Région de Bruxelles-Capitale,5.366845e-07
105,371.0,"Ministère des Affaires étrangères, Commerce ex...",5.366845e-07


* <span style="color:blue">**See table of results**</span>

#### <span style="color:darkviolet">**Q2 Which percentage of the companies are under which Status?**

In [9]:
df_q2 = df_enterprise
df_q2['Status'].value_counts()

AC    1863292
Name: Status, dtype: int64

* <span style="color:blue">**All companies are under Active (AC) Status**</span>

#### <span style="color:darkviolet">**Q3 Which percentage of the companies are which type of entreprise?**

In [10]:
df_q3 = df_enterprise
df_q3['TypeOfEnterprise'].value_counts()

2    1099649
1     763643
Name: TypeOfEnterprise, dtype: int64

In [11]:
df_q3.groupby('TypeOfEnterprise').sum()

/tmp/ipykernel_10762/3048787412.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_q3.groupby('TypeOfEnterprise').sum()


,JuridicalSituation,JuridicalForm,PercentageQ1
TypeOfEnterprise,,,
1,269066,0.0,0.409835
2,3926293,159550652.0,0.590165


* <span style="color:blue">**41% Personne physique**</span>
* <span style="color:blue">**59% Personne morale**</span>

#### <span style="color:darkviolet">**Q4 What is the average company's age in each sector (hint: look what is the NACE code)?**

In [12]:
df_q4 = df_activity

In [13]:
#Lose the Nace 2003 from the table
df_q4 = df_q4.query('NaceVersion == 2008') 

In [14]:
#original rows, just Nace2008
x = 19515324 - 16212783
x

3302541

In [15]:
df_q4_enterprise = df_enterprise.drop(["Status","JuridicalSituation","TypeOfEnterprise","JuridicalForm"], axis=1)
df_q4_enterprise['age'] = pd.Timestamp('now') - (pd.to_datetime(df_q4_enterprise['StartDate'], format='%Y-%m-%d %H:%M:%S'))
df_q4_enterprise['age'] = df_q4_enterprise['age'].astype("timedelta64[D]")/ 365.2425
df_q4_enterprise = df_q4_enterprise.drop(['StartDate','PercentageQ1'], axis=1)
df_q4_enterprise = df_q4_enterprise.rename(columns={'EnterpriseNumber':	'EntityNumber','age':'age'})

In [16]:
df_q4 = pd.merge(df_q4, df_q4_enterprise,
    how='left', on='EntityNumber')

In [17]:
df_q4['age'].isna().sum()

13214541

In [18]:
x = 13214541 / 16212783

In [19]:
df_q4 = df_q4.dropna(subset='age')

In [20]:
df_q4

,EntityNumber,ActivityGroup,NaceVersion,NaceCode,Classification,age
0,0200.065.765,6,2008,84130,MAIN,62.213461
1,0200.065.765,1,2008,41101,MAIN,62.213461
2,0200.068.636,6,2008,36000,MAIN,99.692670
3,0200.068.636,1,2008,36000,MAIN,99.692670
4,0200.068.636,1,2008,93110,SECO,99.692670
...,...,...,...,...,...,...
2998239,0998.534.826,6,2008,63910,MAIN,24.816389
2998240,0998.549.771,6,2008,35110,MAIN,222.816348
2998241,0998.562.738,6,2008,62010,MAIN,23.573379
2998242,0998.576.495,1,2008,82990,MAIN,21.818381


In [21]:
df_q4['Nace_xx'] = df_q4['NaceCode'].apply(lambda x : str(x)[:2])
df_q4['Nace_xx']

0          84
1          41
2          36
3          36
4          93
           ..
2998239    63
2998240    35
2998241    62
2998242    82
2998243    63
Name: Nace_xx, Length: 2998242, dtype: object

In [22]:
df_Nace2008 = df_code.loc[(df_code['Category'] == 'Nace2008') & (df_code['Language'] == 'FR')]
df_Nace2008 = df_Nace2008.drop(columns=['Category','Language'],axis=1)
df_Nace2008 = df_Nace2008.loc[(df_Nace2008['Code'].str.len() == 2)]
df_Nace2008 = df_Nace2008.rename(columns={'Code':'Nace_xx','Description':'Description'})
df_Nace2008

,Nace_xx,Description
8276,01,"Culture et production animale, chasse et servi..."
8524,02,Sylviculture et exploitation forestière
8562,03,Pêche et aquaculture
8596,05,Extraction de houille et de lignite
8614,06,Extraction d'hydrocarbures
...,...,...
14674,95,Réparation d'ordinateurs et de biens personnel...
14746,96,Autres services personnels
14852,97,Activités des ménages en tant qu'employeurs de...
14860,98,Activités indifférenciées des ménages en tant ...


In [23]:
df_q4 = pd.merge(df_q4, df_Nace2008,
    how='left', on='Nace_xx')

In [24]:
df_q4 = df_q4.groupby('Nace_xx').mean().sort_values(by='age',ascending=False)

/tmp/ipykernel_10762/3999027158.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_q4 = df_q4.groupby('Nace_xx').mean().sort_values(by='age',ascending=False)


In [25]:
df_q4 = pd.merge(df_q4, df_Nace2008,
    how='left', on='Nace_xx')

In [26]:
df_q4

,Nace_xx,ActivityGroup,NaceVersion,NaceCode,age,Description
0,84,4.478044,2008.0,84153.246174,39.419530,Administration publique et défense; sécurité s...
1,36,1.286624,2008.0,36000.000000,33.403390,"Captage, traitement et distribution d'eau"
2,99,5.023529,2008.0,87469.411765,28.664662,Activités des organisations et organismes extr...
3,94,4.157523,2008.0,94739.340801,28.449566,Activités des organisations associatives
4,19,1.555556,2008.0,19195.238095,28.416911,Cokéfaction et raffinage
...,...,...,...,...,...,...
76,82,1.104015,2008.0,82691.308046,8.022065,Services administratifs de bureau et autres ac...
77,85,1.246770,2008.0,85564.844061,7.954835,Enseignement
78,53,1.438962,2008.0,53183.300723,7.722184,Activités de poste et de courrier
79,63,1.078067,2008.0,63400.164715,7.675516,Services d'information


* <span style="color:blue">**See table above**</span>